In [1]:
# -*- coding: utf-8 -*-
import os, time, warnings, urllib.parse
import requests, pandas as pd, urllib3

warnings.filterwarnings("ignore")
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

SAVE_DIR = "水果資料"
YEAR_START = 2020
YEAR_CUTOFF = 2025
FRUITS = ["蘋果","桃","葡萄","香蕉","芒果","木瓜","鳳梨","草莓","藍莓","覆盆子"]

FRUIT_UNIT = 135   # 果品
VEG_UNIT   = 113   # 蔬菜

HEADERS = {"User-Agent": "Mozilla/5.0 (MOA-like/1.0)", "Accept": "application/json, text/plain, */*"}
TIMEOUT = 30

# 欄位異名容錯
COL_MAP = {"年份":"年度","果品名稱":"果品類別","蔬菜名稱":"蔬菜類別"}

def route_unit_and_col(name: str):
    return (VEG_UNIT, "蔬菜類別") if name == "草莓" else (FRUIT_UNIT, "果品類別")

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty: return df
    return df.rename(columns={c: COL_MAP.get(c, c) for c in df.columns})

def is_json_array(text: str) -> bool:
    return bool(text) and text.lstrip().startswith("[")

def fetch(url: str):
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT, verify=False)
    r.raise_for_status()
    if not is_json_array(r.text):
        return []
    return r.json()

def pull_one(name: str) -> pd.DataFrame:
    unit, cat_col = route_unit_and_col(name)
    frames = []

    for year in range(YEAR_START, YEAR_CUTOFF):
        # ★ 關鍵：使用 like（不加引號），與你貼的網址一致
        # 例：https://data.moa.gov.tw/... ?UnitId=135&$filter=年度 like 2012 and 果品類別 like 蘋果
        filter_str = f"年度 like {year} and {cat_col} like {name}"
        url = ("https://data.moa.gov.tw/Service/OpenData/DataFileService.aspx"
               f"?UnitId={unit}&$filter={urllib.parse.quote(filter_str, safe='')}")
        data = fetch(url)
        if data:
            frames.append(pd.DataFrame(data))
        time.sleep(0.15)  # 微節流

    if not frames:
        return pd.DataFrame()

    df = normalize_columns(pd.concat(frames, ignore_index=True))

    # 年份安全轉型 + 範圍保險
    if "年度" in df.columns:
        df["年度"] = pd.to_numeric(df["年度"], errors="coerce")
        df = df[(df["年度"] >= YEAR_START) & (df["年度"] <= YEAR_CUTOFF)]

    # ★ 本地精確過濾（避免葡萄柚混進葡萄）
    if cat_col in df.columns:
        df = df[df[cat_col].astype(str) == name]
    else:
        # 萬一後端欄位擺盪，再對一次
        alt = "果品類別" if cat_col == "蔬菜類別" else "蔬菜類別"
        if alt in df.columns:
            df = df[df[alt].astype(str) == name].rename(columns={alt: cat_col})

    if "年度" in df.columns:
        df = df.sort_values("年度").reset_index(drop=True)

    df["查詢水果"] = name
    return df

def main():
    os.makedirs(SAVE_DIR, exist_ok=True)
    summary = []
    xlsx_tabs = {}

    for name in FRUITS:
        route = "蔬菜(113)" if name == "草莓" else "果品(135)"
        df = pull_one(name)
        if df.empty:
            print(f"⚠️ 無資料：{name}（{route}）")
            summary.append({"水果": name, "總筆數": 0, "年份範圍": "", "路由": route})
            continue

        out_csv = os.path.join(SAVE_DIR, f"{name}.csv")
        df.to_csv(out_csv, index=False, encoding="utf-8-sig")
        yrs = df["年度"].dropna().astype(int).tolist() if "年度" in df.columns else []
        yr_range = f"{min(yrs)}–{max(yrs)}" if yrs else ""
        print(f"✅ 已儲存：{out_csv}（{len(df)} 筆，{route}，年份：{yr_range}）")

        xlsx_tabs[name[:20]] = df
        summary.append({"水果": name, "總筆數": len(df), "年份範圍": yr_range, "路由": route})

    # sm = pd.DataFrame(summary)
    # sm_path = os.path.join(SAVE_DIR, "_summary.csv")
    # sm.to_csv(sm_path, index=False, encoding="utf-8-sig")
    # print(f"\n📄 已輸出彙總表：{sm_path}")

    # 一份 Excel（多工作表）
    xlsx_path = os.path.join(SAVE_DIR, "水果彙整.xlsx")
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as w:
        #sm.to_excel(w, index=False, sheet_name="Summary")
        for sheet, dfx in xlsx_tabs.items():
            dfx.to_excel(w, index=False, sheet_name=sheet)
    print(f"📦 已輸出 Excel：{xlsx_path}")

if __name__ == "__main__":
    main()


✅ 已儲存：水果資料\蘋果.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\桃.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\葡萄.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\香蕉.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\芒果.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\木瓜.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\鳳梨.csv（110 筆，果品(135)，年份：2020–2024）
✅ 已儲存：水果資料\草莓.csv（110 筆，蔬菜(113)，年份：2020–2024）
⚠️ 無資料：藍莓（果品(135)）
⚠️ 無資料：覆盆子（果品(135)）
📦 已輸出 Excel：水果資料\水果彙整.xlsx
